# Assignment 1: PWM

## Setup

In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

void clear_gpios(){
    for(int i = 0; i < 8; ++i){
        write_gpio(i,0);
    }
}

## Test code

In [3]:
import asyncio

cond = True

async def generate_pwm_simple(freq_hz, duty):
    period = 1 / freq_hz
    on_time = period * (duty / 100)
    off_time = period * (1 - duty / 100)

    if duty == 0:
        write_gpio(3, 0)
        await asyncio.sleep(period)
    elif duty == 100:
        write_gpio(3, 1)
        await asyncio.sleep(period)
    else:
        write_gpio(3, 1)
        await asyncio.sleep(on_time)
        write_gpio(3, 0)
        await asyncio.sleep(off_time)

async def flash_leds():
    global cond
    while cond:
        await generate_pwm_simple(0.5, 50)

async def get_btns():
    global cond
    while cond:
        await asyncio.sleep(0.01)
        if btns.read() != 0:
            clear_gpios()
            cond = False

async def main():
    clear_gpios()
    try:
        await asyncio.gather(
            flash_leds(),
            get_btns()
        )
    finally:
        clear_gpios()
        print("Done.")
    
asyncio.run(main())

Done.


## RGB code

In [3]:
import asyncio

"""
cond: runs all of the functions
led: chooses led color. default color is red.
    R=3, G=2, B=1
"""

cond = True
led = 3

async def generate_pwm(freq_hz, duty, gpio):
    # period in s
    period = 1/freq_hz
    on_time = period * (duty/100)
    off_time = period * (1 - (duty/100))
    
    if duty == 0:
        write_gpio(gpio, 0)
        await asyncio.sleep(period)
    elif duty == 100:
        write_gpio(gpio, 1)
        await asyncio.sleep(period)
    else:
        write_gpio(gpio, 1)
        await asyncio.sleep(on_time)
        write_gpio(gpio, 0)
        await asyncio.sleep(off_time)    
        
async def flash_leds():
    global cond, led
    while cond:
        start = time.time()
        while cond and time.time() - start < 1.0:
            await generate_pwm(55, 50, led)

        start = time.time()
        while cond and time.time() - start < 1.0:
            await generate_pwm(10, 0, led)
        
async def get_btns():
    global cond, led
    while cond:
        await asyncio.sleep(0.01)
        # turn off led to prepare for transition if any button is pressed
        if btns[0].read() != 0:
            # R
            led = 3
        if btns[1].read() != 0:
            # G
            led = 2
        if btns[2].read() != 0:
            # B
            led = 1
        if btns[3].read() != 0:
            clear_gpios()
            cond = False
            
async def main():
    clear_gpios()
    print("Running program")
    try:
        await asyncio.gather(
            flash_leds(),
            get_btns()
        )
    finally:
        clear_gpios()
        print("Done.")
    
asyncio.run(main())

Running program
Done.
